<a href="https://colab.research.google.com/github/fatemafaria142/Instructions-Tuning-Across-Various-LLMs-with-Alpaca-Dataset/blob/main/Instructions_Tuning_using_Starling_LM_7B_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/tatsu-lab/alpaca?row=0

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("tatsu-lab/alpaca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**
* The dataset contains four columns.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("Text:", data['text'])
    print("\n-----------------------------\n")


Data Point 1:
Instruction: Give three tips for staying healthy.
Input: 
Output: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.
Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.

-----------------------------

Data Point 2:
Instruction: What are the three primary colors?
Input: 
Output: The three primary colors are red, blue, and yellow.
Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are the three primary colors?

### 

### **We will use just a small subset of the data for this training example**

In [5]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(300))

In [6]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 300
    })
})

### **Prompt Creation**

In [7]:
def create_prompt(sample):
    # Use a predefined template for instructions
    prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. "
    prompt += sample['instruction']
    single_turn_prompt = f"GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant: {sample['output']}"
    return single_turn_prompt

In [8]:
create_prompt(instruct_tune_dataset["train"][0])

'GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Give three tips for staying healthy.<|end_of_turn|>GPT4 Correct Assistant: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

In [9]:
create_prompt(instruct_tune_dataset["train"][1])

'GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. What are the three primary colors?<|end_of_turn|>GPT4 Correct Assistant: The three primary colors are red, blue, and yellow.'

In [10]:
create_prompt(instruct_tune_dataset["train"][2])

'GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Describe the structure of an atom.<|end_of_turn|>GPT4 Correct Assistant: An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.'

In [11]:
create_prompt(instruct_tune_dataset["train"][3])

'GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. How can we reduce air pollution?<|end_of_turn|>GPT4 Correct Assistant: There are a number of ways to reduce air pollution, such as shifting to renewable energy sources, encouraging the use of public transportation, prohibiting the burning of fossil fuels, implementing policies to reduce emissions from industrial sources, and implementing vehicle emissions standards. Additionally, individuals can do their part to reduce air pollution by reducing car use, avoiding burning materials such as wood, and changing to energy efficient appliances.'

In [12]:
create_prompt(instruct_tune_dataset["train"][4])

'GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Describe a time when you had to make a difficult decision.<|end_of_turn|>GPT4 Correct Assistant: I had to make a difficult decision when I was working as a project manager at a construction company. I was in charge of a project that needed to be completed by a certain date in order to meet the client’s expectations. However, due to unexpected delays, we were not able to meet the deadline and so I had to make a difficult decision. I decided to extend the deadline, but I had to stretch the team’s resources even further and increase the budget. Although it was a risky decision, I ultimately decided to go ahead with it to ensure that the project was completed on time and that the client’s expectations were met. The project was eventually successfully completed and this was seen as a testament to my leadership and decision-making abilities.'

In [13]:
create_prompt(instruct_tune_dataset["train"][5])

'GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Identify the odd one out.<|end_of_turn|>GPT4 Correct Assistant: Telegram'

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [15]:
mode_id = "berkeley-nest/Starling-LM-7B-alpha"

In [16]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently:**

In [18]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [20]:
prompt = """GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Generate a random password of 6 characters in length using only capital letters and numbers<|end_of_turn|>GPT4 Correct Assistant:"""

In [21]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<s> GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Generate a random password of 6 characters in length using only capital letters and numbers<|end_of_turn|> GPT4 Correct Assistant: Here is a randomly generated password of 6 characters in length using only capital letters and numbers: JJ8#V3B<|end_of_turn|>'

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [26]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [27]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [28]:
from transformers import TrainingArguments
output_model= "Starling_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [29]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [30]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.551900
20,1.117700
30,1.047600
40,1.025400
50,0.977900
60,0.999000
70,0.981400
80,0.976700
90,0.967200
100,0.964100


TrainOutput(global_step=100, training_loss=1.060887565612793, metrics={'train_runtime': 853.4807, 'train_samples_per_second': 0.469, 'train_steps_per_second': 0.117, 'total_flos': 8741776785408000.0, 'train_loss': 1.060887565612793, 'epoch': 0.55})

### **Save the model**

In [31]:
trainer.save_model("Starling_instruct_generation")

In [32]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [33]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [52]:
prompt = "GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. "
prompt += "Explain what is a neural network <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Explain what is a neural network <|end_of_turn|>  GPT4 Correct Assistant: A neural network is an interconnected group of artificial neurons or nodes that can process and perform tasks based on the input it receives from a given dataset. It works similarly to the human brain by mimicking the way neurons in the brain send information to each other. Neural networks are typically used for tasks such as pattern recognition, classification, and regression, and they have the ability to learn and adapt from new input to improve their performance over time.<|end_of_turn|>


### **Example No:2**

In [53]:
prompt = "GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. "
prompt += "Calculate the cost of 4 apples at $2.50 each. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Calculate the cost of 4 apples at $2.50 each. <|end_of_turn|>  GPT4 Correct Assistant: 4 apples at $2.50 each would cost 10.<|end_of_turn|>


### **Example No:3**

In [54]:
prompt = "GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. "
prompt += "Generate a list of business ideas for a food delivery service. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Below is an instruction that describes a task. Write a response that appropriately completes the request. Generate a list of business ideas for a food delivery service. <|end_of_turn|>  GPT4 Correct Assistant: Here are a few business ideas for a food delivery service:

1. Gourmet meal delivery - Offer premium, unique, and gourmet meals to customers who enjoy high-quality and delicious foods.
2. Healthy food delivery - Provide healthy and nutritious meals for people seeking to maintain a healthy lifestyle. This could include meal prep, diet plans, and customizable menu options.
3. Fast-food delivery - Deliver fast food from popular restaurants to customers who want their food quickly and efficiently.
4. Ethnic food delivery - Specialize in delivering authentic and traditional cuisine from various cultures around the world.
5. Plant-based food delivery - Offer a variety of vegan, vegetarian, and plant-based meals that cater to people with dietary restrictions or pr